In [21]:
import pandas as pd
import numbers
pd.options.mode.chained_assignment =default='warn'
import matplotlib.pyplot as plt
from matplotlib.mlab import PCA
import numpy as np
from collections import Counter
import mlpy


# # Read data from dict and csv

# In[2]:

df = pd.read_csv('student-por.csv')
print(df.columns)
df.columns = ['school', 'sex', 'age', 'rural', 'big_family', 'divorced', 'mom_edu', 'dad_edu',
       'mom_job', 'dad_job', 'reason_for_school', 'guardian', 'travel_time', 'study_time',
       'failures', 'extra_school_support', 'extra_family_support', 'paid', 'extracurricular', 'preschool',
       'college', 'internet_access', 'dating', 'home_life', 'free_time', 'party', 'wd_drinker',
       'weekend_drinker', 'health', 'absences', 'fall_grade', 'spring_grade', 'final_grade']
dictionary = pd.read_csv('dict.csv')
dictionary.drop(["Unnamed: 2"], 1)
dictionary['col'] = df.columns


# # Transform data to readable format

# In[3]:

original_data = df.copy()
df = df.replace(['yes', 'no'], [1,0])
df.divorced = df.divorced.replace(["A", "T"], [1,0])
df.big_family = df.big_family.replace(["LE3", "GT3"], [0,1])
df.rural = df.rural.replace(["U","R"], [0,1])
df.sex = df.sex.replace(["M","F"], [0,1])
df.school = df.school.replace(["GP","MS"], [1,2])
df.guardian = df.guardian.replace(["mother","father", "other"], [1,2,3])
df.mom_job = df.mom_job.replace(["teacher","health","services","at_home","other"], [1,2,3,4,5])
df.dad_job = df.dad_job.replace(["teacher","health","services","at_home","other"], [1,2,3,4,5])
df.reason_for_school = df.reason_for_school.replace(["home","reputation", "course","other"], [1,2,3,4])
df['index'] = df.index
df["risk_score"] = df.wd_drinker * df.weekend_drinker


df['school_GP'] = df.school.replace([1,2], [1,0])#["GP","MS"], [1,2])
df['school_MS'] = df.school.replace([1,2], [0,1])

df['guard_mom'] = df.guardian.replace([1,2,3], [1,0,0])#["mother","father", "other"], [1,2,3])
df['guard_dad'] = df.guardian.replace([1,2,3],[0,1,0])#["mother","father", "other"], [1,2,3])
df['guard_other'] = df.guardian.replace([1,2,3], [0,0,1])#["mother","father", "other"], [1,2,3])

#["teacher","health","services","at_home","other"], [1,2,3,4,5])
df['mom_teach'] = df.mom_job.replace([1,2,3,4,5], [1,0,0,0,0])
df['mom_health'] = df.mom_job.replace([1,2,3,4,5], [0,1,0,0,0])
df['mom_service'] = df.mom_job.replace([1,2,3,4,5], [0,0,1,0,0])
df['mom_home'] = df.mom_job.replace([1,2,3,4,5], [0,0,0,1,0])
df['mom_other'] = df.mom_job.replace([1,2,3,4,5], [0,0,0,0,1])

df['dad_teach'] = df.dad_job.replace([1,2,3,4,5], [1,0,0,0,0])
df['dad_health'] = df.dad_job.replace([1,2,3,4,5], [0,1,0,0,0])
df['dad_service'] = df.dad_job.replace([1,2,3,4,5], [0,0,1,0,0])
df['dad_home'] = df.dad_job.replace([1,2,3,4,5], [0,0,0,1,0])
df['dad_other'] = df.dad_job.replace([1,2,3,4,5], [0,0,0,0,1])

#df.reason_for_school = df.reason_for_school.replace(["home","reputation", "course","other"], [1,2,3,4])
df['reason_home'] = df.reason_for_school.replace([1,2,3,4], [1,0,0,0])
df['reason_rep'] = df.reason_for_school.replace([1,2,3,4], [0,1,0,0])
df['reason_course'] = df.reason_for_school.replace([1,2,3,4], [0,0,1,0])
df['reason_other'] = df.reason_for_school.replace([1,2,3,4], [0,0,0,1])


# # Build dictionary for translation

# In[4]:

dictionary.columns = ['original_name', 'meaning', 'classes', 'col']
dictionary = dictionary.replace(np.nan, '', regex=True)
dictionary  = dictionary.set_value(5, 'classes', "1 = divorced")
dictionary  = dictionary.set_value(4, 'classes', "1 = >3")
dictionary  = dictionary.set_value(3, 'classes', "1 = rural")
dictionary  = dictionary.set_value(1, 'classes', "0=male, 1=female")
dictionary  = dictionary.set_value(0, 'classes', "1=GP 1=MS")
dictionary  = dictionary.set_value(11, 'classes', "1=mother, 2=father, 3=other")
dictionary  = dictionary.set_value(8, 'classes', "1=teacher, 2=health 3=services 4=at_home 5=other")
dictionary  = dictionary.set_value(9, 'classes', "1=teacher, 2=health 3=services 4=at_home 5=other")
dictionary  = dictionary.set_value(10, 'classes', "1=close to home, 2=reputation 3=course4 =other")


Index(['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
       'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime',
       'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
       'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc',
       'Walc', 'health', 'absences', 'G1', 'G2', 'G3'],
      dtype='object')


In [52]:
cols = [ 'sex', 'age', 'rural', 'big_family', 'divorced', 'mom_edu',
       'dad_edu','travel_time', 'study_time', 'failures', 'extra_school_support',
       'extra_family_support', 'paid', 'extracurricular', 'preschool',
       'college', 'internet_access', 'dating', 'home_life', 'free_time',
       'party', 'health', 'absences',
       'fall_grade', 'spring_grade', 'final_grade',
       'school_GP', 'school_MS', 
       'guard_mom', 'guard_dad', 'guard_other',
       'mom_teach', 'mom_health', 'mom_service', 'mom_home', 'mom_other',
       'dad_teach', 'dad_health', 'dad_service', 'dad_home', 'dad_other',
       'reason_home', 'reason_rep', 'reason_course', 'reason_other'
       ]

newData = df[cols]
matrix = newData.as_matrix()
print(str(matrix.shape))
# now PCA on the matrix

results = PCA(matrix)

(649, 45)


In [53]:
# print out eigen vector array from PCA
print(results.fracs)


# this shows prop of variance
# print(results.fracs)

# proj = results.project(matrix)
# proj here is the same as results.Y because results.Y shows the values in terms of eigen vectors/values?



[  1.10128370e-01   5.52960871e-02   5.38859579e-02   4.53972950e-02
   3.92231427e-02   3.85602902e-02   3.61535335e-02   3.50233986e-02
   3.36046492e-02   3.03708670e-02   2.86729933e-02   2.85628530e-02
   2.78755528e-02   2.75967140e-02   2.54173309e-02   2.41672890e-02
   2.36828142e-02   2.25428038e-02   2.17342974e-02   2.07885795e-02
   2.06529971e-02   1.99746642e-02   1.96232946e-02   1.82749154e-02
   1.72591736e-02   1.68103412e-02   1.63315083e-02   1.62332830e-02
   1.55132619e-02   1.43337801e-02   1.37835668e-02   1.32889360e-02
   1.29805177e-02   1.25296513e-02   1.18836650e-02   1.10411730e-02
   1.00187388e-02   5.48455751e-03   3.70460981e-03   1.59254568e-03
   2.17189914e-32   7.89032811e-33   6.41907378e-33   4.47225917e-33
   9.57493706e-34]


In [54]:
print(results.Y)
# print("and y" + str(results.Y))

[[ -7.14558925e-01  -1.78103238e+00   1.08374262e+00 ...,  -4.92661467e-16
    6.93889390e-16   1.33226763e-15]
 [ -9.32091727e-01   1.06553917e+00  -8.78324950e-01 ...,   1.06165077e-15
   -4.16333634e-16  -1.83186799e-15]
 [  6.71384782e-02   1.09414493e+00  -1.14759611e+00 ...,  -5.55111512e-17
   -3.33066907e-16   3.33066907e-16]
 ..., 
 [ -2.23838238e+00   1.85205292e+00  -1.35838677e+00 ...,   6.24500451e-17
   -6.38378239e-16   1.22124533e-15]
 [ -1.65861305e+00  -7.51643717e-01   1.79137839e+00 ...,   5.06539255e-16
   -1.41553436e-15   6.66133815e-16]
 [ -1.83238750e+00   7.59113453e-02   5.31011753e-01 ...,   9.50628465e-16
   -3.60822483e-16   1.33226763e-15]]


In [55]:
#print(results.Wt[0])

In [56]:
matrix

array([[ 1, 18,  0, ...,  0,  1,  0],
       [ 1, 17,  0, ...,  0,  1,  0],
       [ 1, 15,  0, ...,  0,  0,  1],
       ..., 
       [ 1, 18,  0, ...,  0,  1,  0],
       [ 0, 17,  0, ...,  0,  1,  0],
       [ 0, 18,  1, ...,  0,  1,  0]])

In [57]:
# create pca class from mlpy
pca = mlpy.PCA()

In [58]:
# feed dataset to it
pca.learn(matrix)

In [59]:
eigens = pca.evals()
# print(results.fracs)
# print(eigens)
      
prop = 0
propVar = results.fracs

k = 20
for x in range(0,k+1):
    prop += propVar[x]
    
print(str(prop))




0.749337816189


In [14]:
# reduce dimensions of data, holding K values
z = pca.transform(matrix, 1)
print(str(z.shape))

(649, 1)


In [16]:
xnew = pca.transform_inv(matrix) # trasform data back to original dimensions 
